In [1]:
!pip install -q bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00


In [2]:
pip install -U bitsandbytes

In [16]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TextStreamer
import torch
from google.colab import userdata
from huggingface_hub import login


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [6]:
GEMMA = "google/gemma-3-27b-it"
PHI = "microsoft/Phi-3-mini-4k-instruct"
LLAMA = "meta-llama/Llama-3.1-8B"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"
QWEN3 = "Qwen/Qwen3-0.6B"

In [7]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {'role': 'user', 'content': 'Tell me a light hearted joke related to Data Scientists.'},
]

In [8]:
quantization_configuration = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4"
  )

In [12]:
tokenizer = AutoTokenizer.from_pretrained(QWEN3, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

model = AutoModelForCausalLM.from_pretrained(QWEN3, quantization_config=quantization_configuration, device_map="auto")


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [13]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 531.5 MB


In [14]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (po

In [15]:
outputs = model.generate(inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell me a light hearted joke related to Data Scientists.<|im_end|>
<|im_start|>
<think>
Okay, the user wants a light-hearted joke related to data scientists. Let me think of something fun. Data scientists use algorithms, right? Maybe something about algorithms being the best in the world. How about a joke about a data scientist who can make everything work, but the algorithm is the one who made it possible. Maybe play on the word "algorithm" and "algorithmic". Something catchy. Maybe a play on "data" and "data scientist". Like, "A


In [19]:
def generate_response_from_open_source_model(model, message):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  streamer = TextStreamer(tokenizer)
  inputs = tokenizer.apply_chat_template(message, return_tensors="pt").to(device)
  model = AutoModelForCausalLM.from_pretrained(model, quantization_config=quantization_configuration, device_map="auto")
  outputs = model.generate(inputs, max_new_tokens=100, streamer=streamer)
  tokenizer.decode(outputs[0])

  del inputs, outputs, model, streamer, tokenizer
  torch.cuda.empty_cache()

In [20]:
generate_response_from_open_source_model(QWEN3, messages)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell me a light hearted joke related to Data Scientists.<|im_end|>
<|im_start|>
<think>
Okay, the user wants a light-hearted joke related to data scientists. Let me think of something fun. Data scientists are people who analyze data, so maybe something about data in a playful way.

How about a joke about data in a data warehouse? Like, "Data scientists are the ones who make data in a data warehouse." Then add a twist. Maybe mention how they use data to make decisions. Oh, and a play on words with "data" and "data warehouse
